In [73]:
import numpy as np
import pandas as pd
import codecs

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import LinearSVC,SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import RussianStemmer
from nltk.corpus import stopwords
from tqdm import tqdm
import pickle
from StemmedCountVectorizer import StemmedCountVectorizer

from sklearn.model_selection import ShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score,f1_score

In [74]:
source_splited_sentences = [];
with codecs.open("1.txt",'r', encoding='utf-8') as file_sentences :
    source_splited_sentences = file_sentences.readlines();
    
ekmtr = np.array([ [sentence.split('\t')[1], sentence.split('\t')[2],sentence.split('\t')[3].replace('\n','')] for sentence in source_splited_sentences ])
df_ekmtr = pd.DataFrame(ekmtr,columns = ['name', 'ekmtr','mtr_type'])
df_ekmtr['ekmtr1'] = df_ekmtr.ekmtr.apply(lambda x: x.split('_')[0])
df_ekmtr['ekmtr2'] = df_ekmtr.ekmtr.apply(lambda x: x.partition('_')[2])
df_ekmtr =  df_ekmtr[df_ekmtr.name !='name']

##########################


#('clf', SVC(kernel='linear', C=1))
stem_vectorizer = StemmedCountVectorizer(stemmer)
text_clf = Pipeline([('vect', stem_vectorizer), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression()) ])


In [75]:
df_ekmtr.shape

(498263, 5)

In [76]:
df_ekmtr  = df_ekmtr.groupby("ekmtr").filter(lambda x: len(x) > 1)
df_ekmtr.shape

(498035, 5)

In [77]:
df_ekmtr.head()

,name,ekmtr,mtr_type,ekmtr1,ekmtr2
1,Эмаль ХВ-785 коричневая ГОСТ 7313-75,115_03_01_09,NULL,115,03_01_09
2,Отвод бесшовный приварной крутоизогнутый 60 21...,111_01_12,NULL,111,01_12
3,"""Кран Bonomi 760005 Ду1"""" ANSI150 CF8M привод ...",101_01_01_05,NULL,101,01_01_05
4,"Клапан КМР ЛГ 301 С 50 1,0 Л НО У в комплектац...",101_01_04_07_04,NULL,101,01_04_07_04
5,"Теплообменник 500 ТУ-2,5-М1/20Г-3-Т-2-У-И ТУ 3...",110_02_01_01,NULL,110,02_01_01


In [80]:
text_clf.fit(df_ekmtr.name.values,df_ekmtr.ekmtr1.values)

Pipeline(steps=[('vect', StemmedCountVectorizer(stemmer=<RussianStemmer>)), ('tfidf', TfidfTransformer(norm=u'l2', smooth_idf=True, sublinear_tf=False,
         use_idf=True)), ('clf', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [81]:
ekclass =  {}
for i in df_ekmtr.ekmtr1.unique():
    print i, df_ekmtr[df_ekmtr.ekmtr1 == i].ekmtr2.unique().shape
    if df_ekmtr[df_ekmtr.ekmtr1 == i].ekmtr2.unique().shape[0] > 1 :
        text_clf_i = Pipeline([('vect', StemmedCountVectorizer(stemmer)), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression()) ])
        text_clf_i.fit(df_ekmtr[df_ekmtr.ekmtr1 == i].name.values,df_ekmtr[df_ekmtr.ekmtr1 == i].ekmtr2.values)
        ekclass[i]= text_clf_i
    

115 (158,)
111 (46,)
101 (92,)
110 (45,)
112 (332,)
107 (124,)
108 (19,)
128 (115,)
105 (244,)
7 (28,)
106 (93,)
114 (199,)
14 (1,)
12 (14,)
103 (374,)
113 (65,)
102 (62,)
104 (57,)
116 (95,)
15 (2,)
109 (85,)
130 (115,)
13 (59,)
8 (14,)
4 (18,)
3 (10,)
5 (13,)
2 (11,)
6 (34,)
16 (1,)
10 (49,)
9 (6,)
18 (12,)
1 (11,)
99 (1,)
17 (1,)
11 (5,)
23 (2,)
120 (5,)
20 (1,)
119 (2,)
600 (1,)
21 (1,)
123 (1,)


In [54]:
def predict_ekmtr(mtr):
    ek1 = text_clf.predict([mtr])[0]
    if df_ekmtr[df_ekmtr.ekmtr1 == ek1].ekmtr2.unique().shape[0] > 1 : 
        ek2 = ekclass[ek1].predict([mtr])[0]
    else : 
        ek2 = df_ekmtr[df_ekmtr.ekmtr1 == ek1].ekmtr2.unique()[0]
    return ek1 + '_'+ ek2
ddd = text_clf.predict(['Подстанция комплектная трансформаторная'])[0]
ekclass[ddd].predict(['Подстанция комплектная трансформаторная КТП-1600/10/0,4-ШЭ-218 УХЛ4 (6048-ESS-001) ARUDTUK00-01'])


array([u'02_02'], dtype=object)

In [55]:
predict_ekmtr('Кольцо уплотнительное Герметика')

u'112_12_01_02'

In [82]:
with open('ek1.pickle', 'wb') as f:
    pickle.dump(text_clf, f)
with open('ekclass.pickle', 'wb') as f:
    pickle.dump(ekclass, f)




In [60]:
text_clf_1111.predict(['Кольцо уплотнительное Герметика'])

array([u'112'], dtype=object)

In [67]:
df_ekmtr[df_ekmtr.mtr_type != 'NULL'].head()

,name,ekmtr,mtr_type,ekmtr1,ekmtr2
8,Уровнемер радарный со стержневым волноводом VE...,108_03_02,ZGEN,108,03_02
11,Сигнализатор уровня вибрационный VEGA VEGASWIN...,108_03_02,ZMAT,108,03_02
12,Каучук синтетический цис-бутадиеновый СКД-ЕФ м...,6_16,ZGEN,6,16
16,Плинт размыкаемый Hyperline KR-PL-10-BRK-0,128_07_04,ZMAT,128,07_04
19,"""Полистирол вспенивающийся """"АЛЬФАПОР"""" ПСВ, т...",7_27,ZGEN,7,27


In [78]:
stem_vectorizer_mtr_type = StemmedCountVectorizer(stemmer)
text_clf_mtr_type = Pipeline([('vect', stem_vectorizer_mtr_type), ('tfidf', TfidfTransformer()), ('clf', LogisticRegression()) ])
text_clf_mtr_type.fit(df_ekmtr[df_ekmtr.mtr_type != 'NULL'].name.values,df_ekmtr[df_ekmtr.mtr_type != 'NULL'].mtr_type.values)
text_clf_mtr_type.predict(['Кольцо уплотнительное Герметика'])
with open('mtr_type.pickle', 'wb') as f:
    pickle.dump(text_clf_mtr_type, f)
with open('df_ekmtr.pickle', 'wb') as f:
    pickle.dump(df_ekmtr, f)


In [79]:
text_clf_mtr_type.predict(['Подстанция комплектная трансформаторная'])

array([u'ZEQU'], dtype=object)